In [1]:
# 1. 데이터 파일 읽기
import pandas as pd

df = pd.read_csv("./data/summary_two.csv")
df.tail()


,docid,content,summary
4267,ae28101b-a42e-45b7-b24b-4ea0f1fb2d50,비뇨기계와 순환계는 혈액이 신장을 통과하면서 폐기물과 물이 제거될 때 관여하는 두 ...,"비뇨기계는 신장과 요관을 통해 혈액 정화와 노폐물 제거에 기여하며, 순환계는 심장과..."
4268,eb727a4f-29c7-4d0c-b364-0e67de1776e9,로봇은 현대 산업에서 많은 역할을 수행할 수 있습니다. 그러나 로봇 사용의 중대한 ...,"로봇은 산업 현장에서 인간의 위험 작업을 수행하며, 조립 과정의 정확성과 일관성 유..."
4269,0c8c0086-c377-4201-81fa-25159e5435a7,"월경은 여성의 생리주기에 따라 발생하는 현상으로, 에스트로겐과 프로게스테론 수치의 ...","월경은 여성의 생리주기와 에스트로겐·프로게스테론 변화에 의해 발생하며, 자궁 내막 ..."
4270,06da6a19-ec78-404e-9640-9fc33f63c6a2,식물이 내뿜는 가스는 산소입니다. 식물은 광합성 과정을 통해 태양 에너지를 이용하여...,"식물은 광합성 과정을 통해 태양 에너지와 이산화탄소를 이용해 산소를 생성하며, 이 ..."
4271,03c36d5e-c711-4dc2-b4db-aaeb94d86395,"버퍼 오버런은 개발자들이 만든 널리 퍼져 있는 앱의 코딩 오류로써, 공격자가 시스템...","버퍼 오버런은 개발자 코딩 오류로 발생하는 보안 취약점으로, 메모리 버퍼 초과 접근..."


In [2]:
# 2. 정보 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4272 entries, 0 to 4271
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   docid    4272 non-null   object
 1   content  4272 non-null   object
 2   summary  4272 non-null   object
dtypes: object(3)
memory usage: 100.3+ KB


In [ ]:
# 3. 디비 생성 클래스

import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.docstore.in_memory import InMemoryDocstore

class ScienceRAG:
    def __init__(self):
        self.embeddings = HuggingFaceEmbeddings(
                                    model_name="Qwen/Qwen3-Embedding-4B",
                                    model_kwargs={"device": "cuda"} ,
                                    encode_kwargs={"normalize_embeddings": True})
        self.vectorstore = FAISS.from_texts(["초기화용 더미 텍스트"], self.embeddings)
        """  # 임베딩 차원 계산
        dimension = len(self.embeddings.embed_query("test"))
    
        # 코사인 유사도를 위한 IndexFlatIP 생성
        index = faiss.IndexFlatIP(dimension)
    
        # 빈 FAISS 벡터 스토어 생성
        self.vectorstore = FAISS(
            embedding_function=self.embeddings,
            index=index,
            docstore=InMemoryDocstore(),
            index_to_docstore_id={},
            normalize_L2=True  # 벡터 정규화로 코사인 유사도 구현
        ) """

        # 요약 체인
        #self.summary_chain = science_summary_chain
        #self.summary_chain = two_step_chain
    
    def add_documents(self, df):
        """문서를 요약하여 벡터DB에 저장"""
        
        texts = df['summary'].tolist()
        metadatas = [
            {
                'docid': row['docid'],
                'content': row['content']
            } 
            for _, row in df.iterrows()
        ]
        ids = df['docid'].tolist()


        # 2. 검색용 요약을 벡터DB에 저장
        self.vectorstore.add_texts(
            texts,
            metadatas=metadatas,
            ids = ids
        )
        
       
    def search(self, query: str, k: int = 3):
        """요약된 내용으로 검색"""
        results = self.vectorstore.similarity_search(query, k=k)
        return results

In [4]:
# 4. 디비 생성 

science_rag = ScienceRAG()

science_rag.add_documents(df)



/tmp/ipykernel_3341190/3421170127.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(
/data/ephemeral/home/work/python/gx-rag/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
# test
query = "직류와 교류 전류의 차이에 대해 알려줘."
docs = science_rag.vectorstore.similarity_search_with_score(query, k=3)
docs

[(Document(id='07ba99c0-c36a-464c-a896-8466cc84c501', metadata={'docid': '07ba99c0-c36a-464c-a896-8466cc84c501', 'content': '직렬 회로에서 전구들은 같은 전류를 흐르게 합니다. 따라서, 한 전구의 전류가 2A라면, 다른 전구의 전류도 2A입니다. 이는 직렬 회로의 특성으로 인해 발생하는 현상입니다. 직렬 회로에서 전류는 모든 부품을 통과하므로, 전류의 크기는 동일합니다. 따라서, 한 전구의 전류가 2A라면, 다른 전구의 전류도 2A입니다.'}, page_content='직렬 회로에서 전류 흐름은 전구와 회로 구성 요소 간의 전기적 특성에 의해 결정되며, 전류 크기 일치는 연결 방식과 전류의 흐름 특성에 의해 발생한다.'),
  np.float32(0.6963271)),
 (Document(id='c59f4bbd-71f6-4073-807f-09e8f0d3efcc', metadata={'docid': 'c59f4bbd-71f6-4073-807f-09e8f0d3efcc', 'content': '와이어가 전류를 흐르는 와이어가 원형 루프의 형태로 굽혀져 있다면, 와이어의 각 부분 주변에는 자기장이 형성됩니다. 이 자기장은 와이어의 평면에 평행하게 분포됩니다. 와이어의 평면을 기준으로 자기장이 형성되므로, 와이어 주변의 자기장은 와이어의 평면에 평행하게 배치됩니다. 이러한 현상은 전류가 흐르는 와이어 주변에 자기장이 형성되는 일반적인 현상으로 알려져 있습니다. 와이어의 평면에 평행한 자기장은 와이어 주변에서 전류의 흐름을 제어하고, 전류의 방향을 결정하는 역할을 합니다. 따라서, 와이어의 평면에 평행한 자기장은 전류의 흐름과 관련된 중요한 요소입니다.'}, page_content='전류 흐름과 자기장 방향은 직교 관계를 가지며, 와이어 평면에 평행한 자기장이 형성된다. 전류의 방향에 따라 자기장 방향이 결정되며, 이는 와이어 주변에서 전류 흐름을 제어하는 핵심 상호작용이다

In [6]:
# 5. 질의문 생성 체인

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama

llm = ChatOllama(model="alibayram/Qwen3-30B-A3B-Instruct-2507")

# 프롬프트 
convertFormat = """
당신은 문자열 포맷 마이그레이션 전문가 입니다. <message> 에서 content 내용만 문자열로 출력합니다. 만일 content가 여러개가 있으면 전체적인 문맥을 파악하여 질문을 만들어서 출력합니다.

<message>
{message}
</message>

[example]

    <message>{{"role": "user", "content": "피를 맑게 하고 몸 속의 노폐물을 없애는 역할을 하는 기관은?"}}</message> 
    output:피를 맑게 하고 몸 속의 노폐물을 없애는 역할을 하는 기관은? 

    <message>{{"role": "user", "content": "이란 콘트라 사건이 뭐야"}}, {{"role": "assistant", "content": "이란-콘트라 사건은 로널드 레이건 집권기인 1986년에 레이건 행정부와 CIA가 적성국이었던 이란에게 무기를 몰래 수출한 대금으로 니카라과의 우익 성향 반군 콘트라를 지원하면서 동시에 반군으로부터 마약을 사들인 후 미국에 판매하다가 발각되어 큰 파장을 일으킨 사건입니다."}}, {{"role": "user", "content": "이 사건이 미국 정치에 미친 영향은?"}}</message>
    output:1986년에 발생한 이란 콘트라 사건이 미국 정치에 미친 영향은? 


output:
[Your output here - NOTHING ELSE]
"""

# 프롬프트 객체 생성
convertFormat_prompt = PromptTemplate(
    input_variables=["message"],
    template=convertFormat
)

# 출력 파서 (문자열)
output_parser = StrOutputParser()

# LCEL 체인 구성
convertFormat_chain = (
    convertFormat_prompt 
    | llm 
    | output_parser
)

In [22]:
# 6. 과학상식 체크 체인

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama


# 프롬프트 
selectYn = """
당신은 세상의 모든 상식과 지식에 정통한 전문가 입니다. 만약 <message> 가 세상의 상식 또는 지식에 관련된 질문이라면  Y 아니라면 N으로 답해주세요. 너에 관하여 물어보는건 세상의 상식 또는 지식에 관련된 질문이 아니야!

<message>
{message}
</message>

당신은 반드시 Y 뜨는 N으로 답해야 합니다.
Answer:
"""

# 프롬프트 객체 생성
selectYn_prompt = PromptTemplate(
    input_variables=["message"],
    template=selectYn
)

# 출력 파서 (문자열)
output_parser = StrOutputParser()

# LCEL 체인 구성
selectYn_chain = (
    selectYn_prompt 
    | llm 
    | output_parser
)

In [8]:
# 7. 답변 생성 체인

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama


# 프롬프트 
answer = """
당신은 과학 상식 전문가 입니다. <message> 의 질문에 대해서 주어진 <reference> 정보를 활용하여 간결하게 답변을 생성합니다.

    - 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답합니다.
    - 한국어로 답변을 생성합니다..

<message>
{message}
</message>

<reference>
{reference}
</reference>

Answer:
"""

# 프롬프트 객체 생성
answer_prompt = PromptTemplate(
    input_variables=["message"],
    template=answer
)

# 출력 파서 (문자열)
output_parser = StrOutputParser()

# LCEL 체인 구성
answer_chain = (
    answer_prompt 
    | llm 
    | output_parser
)

In [9]:
# 8. 데이터 검색 함수

def query_db(message):

    docs = science_rag.vectorstore.similarity_search_with_relevance_scores(message, k=3)

    #filtered_docs = []
    content = []
    docid= []
    reference = []
    
    for doc, score in docs:
        #if score <= 0.8:    
            #filtered_docs.append((doc, score))    
            content.append(doc.metadata['content'])
            docid.append(doc.metadata['docid'])
            reference.append({"score": float(score), "content": doc.metadata['content']})
    return content, docid, reference

In [28]:
# test

message = convertFormat_chain.invoke({"message": '{"role": "user", "content": "python 공부중인데..."}, {"role": "assistant", "content": "네 꼭 필요한 언어라서 공부해 두면 좋습니다."}, {"role": "user", "content": "숫자 계산을 위한 operator 우선순위에 대해 알려줘."}'})
print(message)
result = selectYn_chain.invoke({"message": "니가 대답을 잘해줘서 너무 신나!"})
result

숫자 계산을 위한 Python의 연산자 우선순위에 대해 알려줘.


'N'

In [29]:
# test

from langchain.globals import set_debug

#message = convertFormat_chain.invoke({"message": '{"role": "user", "content": "기억 상실증 걸리면 너무 무섭겠다."}, {"role": "assistant", "content": "네 맞습니다."}, {"role": "user", "content": "어떤 원인 때문에 발생하는지 궁금해."}'})
message = convertFormat_chain.invoke({"message": '복숭아 키우는 노하우좀?'})

response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}
context = {"message":message}

result = selectYn_chain.invoke({"message": message})

if result == "Y":
    context["reference"], response["topk"], response["references"] = query_db(message)
else:
    context["reference"] = ""
    response["topk"] = []
    response["references"] = []

# 전역 디버그 모드 활성화
set_debug(True)

response["answer"] = answer_chain.invoke(context)
print(response["topk"])
print(response["answer"])

set_debug(False)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "message": "복숭아 키우는 노하우는 무엇인가요?",
  "reference": [
    "복숭아나무는 다른 복숭아보다 병에 대한 저항력이 더 큰 복숭아를 생산합니다. 이러한 특정 복숭아를 정확하게 복제하는 방법은 복숭아나무가 무성 생식을 하도록 하는 것입니다. 복숭아나무는 꽃이 핀 후에 수술을 통해 수술한 꽃을 다른 복숭아나무에 이식하여 교배를 진행합니다. 이렇게 교배된 복숭아는 특정 병에 대한 저항력이 더 큰 특성을 가지게 됩니다. 이 방법을 통해 복숭아나무는 더욱 강건한 복숭아를 생산할 수 있습니다. 이러한 복제 방법은 복숭아 생산자들에게 매우 중요한 기술이며, 복숭아의 품질과 수량을 향상시키는 데 큰 도움이 됩니다.",
    "식물의 성장은 중력과 같은 환경 요인에 의해 영향을 받습니다. 중력은 식물의 뿌리와 줄기의 성장 방향을 결정하는 중요한 역할을 합니다. 만약 씨앗이 옆으로 누워서 싹을 틔우고 성장할 수 있다면, 뿌리는 아래로 성장하고, 줄기는 위로 성장할 것입니다. 이는 중력에 의해 뿌리는 땅 속으로 향하고, 줄기는 빛을 받기 위해 위로 성장하는 식물의 생존 전략입니다. 따라서, 씨앗이 옆으로 누워서 싹을 틔우고 성장할 때, 뿌리는 아래로 성장하고, 줄기는 위로 성장할 가능성이 가장 큽니다.",
    "학생이 다음과 같은 단계를 통해 비료가 옥수수에 미치는 영향을 조사합니다. 첫 번째 단계에서는 햇빛이 가득한 곳에서 같은 종류와 양의 흙에 옥수수 개체군을 심습니다. 이렇게 함으로써 실험 환경을 일정하게 유지할 수 있습니다. 두 번째 단계에서는 각 개체군에 다른 브랜드의 비료를 같은 양만큼 추가합니다. 이렇게 함으로써 비료의 종류를 독립 변수로 설정할 수 있습니다. 세 번째 단계에서는 매일 5분 동안 식물에 물을 줍니다. 이렇게 함으로써 모든 개체군이 동일한 환경에서 성장할 수 있습니다. 네 번째 단계에서

In [ ]:
# test

content, docid, reference = query_db("세제의 거품이 만들어지는 원리는?")
print(content)
print(docid)
print(reference)

In [30]:
# 9. 메인 로직

import json

# 답변 데이터 생성
def answer_question(messages):
    # 함수 출력 초기화
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}

    message = convertFormat_chain.invoke({"message": messages})
    result = selectYn_chain.invoke({"message": message})

    context = {"message":message}

    if result == "Y":
        context["reference"], response["topk"], response["references"] = query_db(message)
    else:
        context["reference"] = ""
        response["topk"] = []
        response["references"] = []
    
    response["answer"] = answer_chain.invoke(context)

    return response


# 답변 저장
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as f, open(output_filename, "w") as of:
        idx = 0
        for line in f:

            #if idx == 10: break
            
            j = json.loads(line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            print(f'Answer: {response["answer"]}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": j["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1


In [31]:
# 10. 메인 

# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
eval_rag("./data/eval.jsonl", "./data/sample_submission.csv")

Test 0
Question: [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]
Answer: 나무의 분류를 조사하기 위해서는 나무의 잎, 꽃, 성장 속도, 온도 범위, 크기 등의 특징을 관찰하고 비교하는 방법이 있습니다. 특히, 같은 속에 속한 나무들은 유사한 특징을 가지므로, 이러한 공통된 특징을 기준으로 분류할 수 있습니다. 또한, 잎의 크기 등은 자를 이용해 정확히 측정하여 분석에 활용할 수 있습니다.

Test 1
Question: [{'role': 'user', 'content': '각 나라에서의 공교육 지출 현황에 대해 알려줘.'}]
Answer: 주어진 참고 자료에서는 각 나라의 공교육 지출 현황에 대한 구체적인 수치나 비교 정보가 포함되어 있지 않습니다. 전세계 공공 교육 지출이 GDP의 약 4%를 차지한다는 정보는 있으나, 특정 국가별 지출 현황은 언급되어 있지 않습니다. 따라서 정보가 부족하여 각 나라의 공교육 지출 현황에 대해 정확히 답변할 수 없습니다.

Test 2
Question: [{'role': 'user', 'content': '기억 상실증 걸리면 너무 무섭겠다.'}, {'role': 'assistant', 'content': '네 맞습니다.'}, {'role': 'user', 'content': '어떤 원인 때문에 발생하는지 궁금해.'}]
Answer: 기억 상실증은 대뇌의 기능 장애로 인해 발생할 가능성이 가장 높습니다. 대뇌는 기억을 조절하는 중요한 부분으로, 그 구조나 기능이 원활하지 않으면 기억 상실이 생길 수 있습니다. 특히 노화로 인한 뇌 기능 저하, 신경 세포의 처리 속도 저하, 뇌의 자원 감소 등이 기억 상실의 원인이 될 수 있습니다.

Test 3
Question: [{'role': 'user', 'content': '통학 버스의 가치에 대해 말해줘.'}]
Answer: 통학 버스는 학생들에게 안전하고 편리한 이동 수단을 제공하며, 학교의 상징이 되기도 합